In [1]:
from json import loads
from mastodon import Mastodon
from numpy import exp, log10
from pandas import read_csv, concat
from pathlib import Path
from scipy.stats import lognorm

from fediverse_analysis.instance_data.analyze import Analyzer

In [2]:
# To Remove
INSTANCES_TO_REMOVE = ['blips2.club', 'epsilon.social', 'fedibird.com', 'mastodon.cl', 'mental.social', 'penguicon.social', 'toot.rebel.ar']
# Input
INSTANCE_DATA_PATH = Path('/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/fedi_data/2023-12-20/07.jsonl')
SAMPLED_INSTANCES_PATH = Path('/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/sample/03/instances.txt')
## List of instances removed during sampling because the timeline was not crawlable
REMOVED_INSTANCES_PATH = Path('/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/sample/01/instances_removed_during_sampling.json')
# Output
RESAMPLE_OUTPUT_FILE_NAME = 'instances_resample'
RESAMPLE_OUTPUT_FILE_EXT = 'txt'
NEW_SAMPLE_OUTPUT_FILE_NAME = 'instances.txt'

In [3]:
# Load data
with open(INSTANCE_DATA_PATH, 'r') as file:
    an = Analyzer(file)
with open(REMOVED_INSTANCES_PATH, 'r') as file:
    removed_instances = loads(file.readline())
with open(SAMPLED_INSTANCES_PATH, 'r') as file:
    sampled_instances = [instance.strip() for instance in file.readlines()]

Number of fediverse instances in input file: 22178
Removed for (partially) no data: 11822
↳ Almost all of these instances run fediverse software other than Mastodon, some run Mastodon with a non-public API.
Removed duplicates: 2
Remaining: 10354


In [4]:
COLUMNS = ['total_users', 'monthly_users', 'total_statuses',
        'mean_weekly_statuses', 'mean_weekly_logins', 'mean_weekly_registrations']

# Prepare sample data
cols_prob_measures = {
    col: lognorm
    for col in COLUMNS
}
df = an.df
## Estimate probability distributions over activity columns
distributions = {
    col: dist.fit(df[col])
    for col, dist in cols_prob_measures.items()
}
## Compute normalize activity score by dividing by the estimated probability.
for col, dist in cols_prob_measures.items():
    shape, location, scale = distributions[col]
    df[f"{col}_log_probability"] = dist.logpdf(df[col], shape, location, scale)
## Compute joint probability (under assumption of independence; using log probabilities for numerical stability)
df["log_probability"] = 0
for col in cols_prob_measures.keys():
    df["log_probability"] += df[f"{col}_log_probability"]

df.sort_values("log_probability", inplace=True)
df.drop(removed_instances, inplace=True)
df["weight"] = exp(-df["log_probability"])

/home/tschaeggaer/Projekte/thesis-ernst/code/venv/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6120: RuntimeWarning: overflow encountered in divide
  return np.sum((1 + np.log(shifted/scale)/shape**2)/shifted)
/home/tschaeggaer/Projekte/thesis-ernst/code/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/tschaeggaer/Projekte/thesis-ernst/code/venv/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6111: RuntimeWarning: invalid value encountered in log
  lndata = np.log(data - loc)


In [5]:
old_sample = df.loc[sampled_instances]
# Remove non-crawlable instances
new_sample = old_sample.drop(INSTANCES_TO_REMOVE, errors='ignore')
df.drop(INSTANCES_TO_REMOVE, inplace=True, errors='ignore')
# Remove already sampled instances
df.drop(sampled_instances, inplace=True, errors='ignore')
# Find actually dropped instances
to_resample = old_sample[~old_sample.index.isin(new_sample.index)]
to_resample

,total_users,monthly_users,total_statuses,mean_weekly_statuses,mean_weekly_logins,mean_weekly_registrations,total_users_log_probability,monthly_users_log_probability,total_statuses_log_probability,mean_weekly_statuses_log_probability,mean_weekly_logins_log_probability,mean_weekly_registrations_log_probability,log_probability,weight
instance,,,,,,,,,,,,,,
mastodon.cl,68,13,1987,25.0,9.5,0.25,-6.531107,-8.941717,-9.395784,-10.245245,-8.663747,-8.168164,-51.945766,3.628769e+22


In [6]:
orig_df = df.copy()
new_instances = []

# Resample by choosing the instance with the weight closest to the dropped instance
for instance in to_resample.index:
    new_instance = log10(df['weight'].div(to_resample.loc[instance]['weight'])).abs().sort_values().idxmin()
    new_instances.append(new_instance)
    df.drop(new_instance, inplace=True)

resample = orig_df.loc[new_instances]
resample

/home/tschaeggaer/Projekte/thesis-ernst/code/venv/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,total_users,monthly_users,total_statuses,mean_weekly_statuses,mean_weekly_logins,mean_weekly_registrations,total_users_log_probability,monthly_users_log_probability,total_statuses_log_probability,mean_weekly_statuses_log_probability,mean_weekly_logins_log_probability,mean_weekly_registrations_log_probability,log_probability,weight
instance,,,,,,,,,,,,,,
zh.pp.social,7,7,64,16.75,5.25,1.0,-3.596011,-8.321744,-8.703085,-9.843967,-8.069782,-9.567844,-48.102434,7.773588e+20


In [7]:
# Save new, complete instance sample to file
new_instance_sample = concat((new_sample, resample)).sort_values('instance')
new_instance_sample.reset_index()['instance'].to_csv(Path(NEW_SAMPLE_OUTPUT_FILE_NAME), index=False, header=False)
# Full DataFrame. Maybe we want to have that data later.
resample.to_csv(Path(RESAMPLE_OUTPUT_FILE_NAME + '_full_data.csv'))
# Raw instance list only.
resampled_instances = resample.reset_index()['instance']
resampled_instances.to_csv(Path(RESAMPLE_OUTPUT_FILE_NAME + '.' + RESAMPLE_OUTPUT_FILE_EXT), index=False, header=False)

In [8]:
# Test if instances are crawlable
for instance in resampled_instances:
    m = Mastodon(api_base_url=instance)
    assert len(m.timeline(timeline='public')) > 0